In [4]:
import boto3
import sagemaker
from sagemaker.xgboost.estimator import XGBoost

In [5]:
# Set region, boto3 and SageMaker SDK variables¶

#You can change this to a region of your choice
import sagemaker
region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

sagemaker_role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity()["Account"]

Using AWS Region: us-east-1


In [11]:
%store

Stored variables and their in-db values:
data_prefix                 -> 'sagemaker-tutorial/data'
default_bucket              -> 'sagemaker-us-east-1-367158743199'
hyperparameters             -> {'max_depth': '3', 'eta': '0.2', 'objective': 'bin
model_data                  -> 's3://sagemaker-us-east-1-367158743199/tf2-resnet-
prefix                      -> 'sagemaker-tutorial'
s3_raw_data                 -> 's3://sagemaker-us-east-1-367158743199/sagemaker-t


# Traina model using XGBoost

 
<span style="color:red">**TODO:  XGBoost details**</span>


## Set the hyperparameters

 
<span style="color:red">**TODO:  XGBoost hyperparameters details**</span>


In [7]:
hyperparameters = {
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


## Create and fit the estimator

In [13]:
estimator_output_path = f's3://{default_bucket}/{prefix}/training_jobs'
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = sagemaker_role,
    instance_count = train_instance_count,
    instance_type = train_instance_type,
    framework_version = "1.0-1")

In [14]:
if 'training_job_1_name' not in locals():
    
    xgb_estimator.fit(inputs = {'train': train_data_uri})
    training_job_1_name = xgb_estimator.latest_training_job.job_name
    %store training_job_1_name
    
else:
    print(f'Using previous training job: {training_job_1_name}')

NameError: name 'train_data_uri' is not defined